In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [12]:
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def save_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)

    anim.save('RL-cartpole.mp4')
    display(display_animation(anim, default_mode='loop'))


In [13]:
# 学習に使う変数を整理
ENV = 'CartPole-v0'
# 状態をどれくらいの値で分割して離散値とするか
NUM_DIGITIZED = 6
# 報酬割引率
GAMMA = 0.99
# 学習率
ETA = 0.5
# 1試行（1エピソード）の最大ステップ数
MAX_STEP = 200
# 最大試行回数（エピソード数）
NUM_EPISODES = 1000


In [14]:
# エージェントクラス
class Agent:

    # 初期化関数で行動数と行動の種類を受けとり，行動を決定するためのBrainクラスを生成する
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)

    # Q関数を更新
    def update_q_function(self, observation, action, reward, observation_next):
        self.brain.update_Q_table(observation, action, reward, observation_next)

    # 行動の決定
    def get_action(self, observation, step):
        return self.brain.decide_action(observation, step)


# エージェントが行う行動を与えられた状態によって判断する部分（強化学習を行う部分）
class Brain:

    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        # Qテーブルを作成。行数は状態を分割数^（4変数）にデジタル変換した値、列数は行動数を示す
        self.q_table = np.random.uniform(low=0, high=1, size=(NUM_DIGITIZED**num_states, num_actions))


    # カート情報を離散化するための閾値を求める
    def bins(self, clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num + 1)[1:-1]

    # カートの情報を離散化する
    def digitize_state(self, observation):
        '''観測したobservation状態を、離散値に変換する'''
        cart_pos, cart_v, pole_angle, pole_v = observation
        digitized = [
            np.digitize(cart_pos, bins=self.bins(-2.4, 2.4, NUM_DIGITIZED)),
            np.digitize(cart_v, bins=self.bins(-3.0, 3.0, NUM_DIGITIZED)),
            np.digitize(pole_angle, bins=self.bins(-0.5, 0.5, NUM_DIGITIZED)),
            np.digitize(pole_v, bins=self.bins(-2.0, 2.0, NUM_DIGITIZED))
        ]
        return sum([x * (NUM_DIGITIZED**i) for i, x in enumerate(digitized)])

    # QテーブルをQ学習により更新
    def update_Q_table(self, observation, action, reward, observation_next):
        # 状態を離散化
        state = self.digitize_state(observation)
        state_next = self.digitize_state(observation_next)
        
        # Qテーブルの最大値を取得
        max_Q_next = max(self.q_table[state_next][:])
        
        # Qテーブルの更新
        self.q_table[state, action] = self.q_table[state, action] + ETA * (reward + GAMMA * max_Q_next - self.q_table[state, action])


    # ε-greedy法で徐々に最適行動を行うようにする
    def decide_action(self, observation, episode):
        state = self.digitize_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))
        
        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.q_table[state][:])
        else:
            action = np.random.choice(self.num_actions)
               
        return action


In [10]:
## CartPoleを実行する環境
class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態を取得
        num_actions = self.env.action_space.n  # CartPoleの行動を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

    ## 実行関数
    def run(self):

        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        is_episode_final = False  # 最終試行フラグ
        frames = []  # 動画用に画像を格納する変数

        # 全エピソードループ
        for episode in range(NUM_EPISODES):
            # エピソード毎に環境を初期化
            observation = self.env.reset()
            
            for step in range(MAX_STEP):
                if is_episode_final:
                    frames.append(self.env.render(mode='rgb_array'))
                    
                # 最初の行動を決める
                action = self.agent.get_action(observation, episode)
                
                # 最初の行動から次の状態を求める
                observation_next, _, done, _ = self.env.step(action)
                
                # 報酬を与える
                if done:
                    if step < 195:
                        # こけたら報酬-1
                        reward = -1
                        complete_episodes = 0
                    else:
                        # 立ったまま終了した場合は報酬1
                        reward = 1
                        complete_episodes += 1
                else:
                    # 途中の報酬は0
                    reward = 0
                    
                self.agent.update_q_function(observation, action, reward, observation_next)
                
                # 状態を次の状態に更新
                observation = observation_next
                    
                # エピソード終了時    
                if done:
                    print('{0}エピソード: {1}ステップで終了'.format(episode, step + 1))
                    break
            
            # 最終エピソードの場合は動画を保存
            if is_episode_final:
                save_as_gif(frames)
                break
            
            # 10回連続で成功したら、次のエピソードで終わりにする
            if complete_episodes >= 10:
                print('10回連続成功')
                is_episode_final = True
                
            
                



    # 最終試行では動画を保存と描画


    # 10エピソード連続成功なら次の試行を描画を行う最終試行とする



In [ ]:
# main
cartpole_env = Environment()
cartpole_env.run()